In [91]:
import json
from pathlib import Path

In [92]:
BASE_DIR = Path(".").resolve()
print(BASE_DIR)

/Users/sandylin/Challenge in CL copy


In [93]:
Data_Path = BASE_DIR / "generator_reference"/"reference.jsonl"
def read_data(dataset:str):
    path = BASE_DIR / "generator_reference" / dataset
    with open(path, "r") as file:
        return file.readlines()

In [94]:
dataset = read_data("reference.jsonl")
print(json.loads(dataset[1]))

{'conversation_id': 'dd6b6ffd177f2b311abe676261279d2f', 'task_id': 'dd6b6ffd177f2b311abe676261279d2f<::>2', 'task_type': 'rag', 'turn': '2', 'dataset': 'MT-RAG Authors (Internal)', 'contexts': [{'document_id': '822086267_7384-8758-0-1374', 'text': "2017 Arizona Cardinals season\nWeek Date Opponent Result Record Game site NFL.com recap September 10 at Detroit Lions L 23 -- 35 0 -- 1 Ford Field Recap September 17 at Indianapolis Colts W 16 -- 13 ( OT ) 1 -- 1 Lucas Oil Stadium Recap September 25 Dallas Cowboys L 17 -- 28 1 -- 2 University of Phoenix Stadium Recap October 1 San Francisco 49ers W 18 -- 15 ( OT ) 2 -- 2 University of Phoenix Stadium Recap 5 October 8 at Philadelphia Eagles L 7 -- 34 2 -- 3 Lincoln Financial Field Recap 6 October 15 Tampa Bay Buccaneers W 38 -- 33 3 -- 3 University of Phoenix Stadium Recap 7 October 22 at Los Angeles Rams L 0 -- 33 3 -- 4 Twickenham Stadium ( London , England ) Recap 8 Bye 9 November 5 at San Francisco 49ers W 20 -- 10 4 -- 4 Levi 's Stadium

In [95]:
data_dict = {}
for data in dataset:
    obj = json.loads(data)
    conv_id = obj["conversation_id"]
    turn = int(obj["turn"])
    context = [(ctx["text"], float(ctx["score"])) for ctx in obj["contexts"]]
    history = [(cov["speaker"],cov["text"]) for cov in obj["input"]][:-1]
    current =  [cov["text"] for cov in obj["input"]][-1]
    target = obj["targets"][0]["text"]
    question_type = obj["Answerability"]
    data_dict[(conv_id, turn)] = {"context": context, "history": history, "current": current, "target": target, "question_type": question_type}

print(json.dumps(data_dict[("dd6b6ffd177f2b311abe676261279d2f", 3)], indent= 4, ensure_ascii= False))
print(len(data_dict))

{
    "context": [
        [
            "Illinois\nThe NFL 's Arizona Cardinals , who currently play in the Phoenix suburb of Glendale , Arizona , played in Chicago as the Chicago Cardinals , until moving to St. Louis , Missouri after the 1959 season . An NBA expansion team known as the Chicago Packers in 1961 -- 1962 , and as the Chicago Zephyrs the following year , moved to Baltimore after the 1962 -- 1963 season . The franchise is now known as the Washington Wizards . Professional",
            17.550575
        ],
        [
            "History of the National Football League\nTwo charter members , the Chicago Cardinals ( now the Arizona Cardinals ) and the Decatur Staleys ( now the Chicago Bears ) , are still in existence . The Green Bay Packers franchise , founded in 1919 , is the oldest team not to change locations , but did not begin league play until 1921 . The New York Football Giants joined in 1925 , followed by the Portsmouth Spartans in 1930 , relocating to Detroit in 193

In [105]:
import random

random.seed(42)

all_keys = list(data_dict.keys())
random.shuffle(all_keys)

train_ratio = 0.8
split_idx = int(len(all_keys) * train_ratio)

train_keys = all_keys[:split_idx]
dev_keys   = all_keys[split_idx:]

# 產生 train / dev dict
train_data = {k: data_dict[k] for k in train_keys}
dev_data   = {k: data_dict[k] for k in dev_keys}

print("Train size:", len(train_data))
print(type(train_data))
print("Dev size:", len(dev_data))

Train size: 673
<class 'dict'>
Dev size: 169


In [97]:
answerable = []
partial = []
unanswerable = []
conversational = []

for k, v in train_data.items():
    qt = v["question_type"]   # 或你自己的欄位
    if qt == ["ANSWERABLE"]:
        answerable.append(v)
    elif qt == ["PARTIAL"]:
        partial.append(v)
    elif qt == ["UNANSWERABLE"]:
        unanswerable.append(v)
    elif qt == ["CONVERSATIONAL"]:
        conversational.append(v)
len(answerable), len(partial), len(unanswerable), len(conversational)

(573, 45, 47, 8)

In [99]:
import random
random.seed(42)

training_data = []

# Answerable：不要全放，避免壓過其他類型
training_data += random.sample(answerable, k=min(450, len(answerable)))

# Partial：允許重複
training_data += random.choices(partial, k=200)

# Unanswerable：允許重複
training_data += random.choices(unanswerable, k=150)

training_data += random.choices(conversational, k=24)

random.shuffle(training_data)

print("Final training size:", len(training_data))
print(training_data[0])

Final training size: 824
{'context': [("Whistleblower Portal | New York State Attorney General\nFile a Complaint\n\nHow can we help you?\n\nI Want To...\n\nAppeal a background check\n\nApply for a Job/Internship\n\nDo Business with the OAG\n\nFile a Complaint\n\nFind Legal Services\n\nKnow My Rights\n\nMake a FOIL Request\n\nRegister/Submit a Filing\n\nReport a Data Breach\n\nRequest a Publication\n\nServe Papers on OAG\n\nUse the Whistleblower Portal\n\nHome\n\n Whistleblower Portal\n \n\nOffice of the New York State Attorney General Whistleblower Portal\n\nThe Office of the New York State Attorney General (OAG) welcomes and encourages whistleblowers, tips, and complaints. Whistleblowers are essential to fulfilling our mission: protecting the citizens of New York. Whistleblowing insiders are particularly critical in opaque industries, where transparency is limited.\nWe take special care to protect whistleblowers. While we encourage whistleblowers to identify themselves, we recognize t

In [110]:
def dump_jsonl(data_dict, path):
    with open(path, "w") as file:
        for val in data_dict:
            history = val["history"]
            current = val["current"]
            target = val["target"]
            context = val["context"]
            question_type = val["question_type"]
            obj = {
                "instruction": "Answer the user's question using ONLY the reference documents.",
                "input":{
                "history": history,
                "current": current,
                "reference": [
                    {"text": doc, "score": float(score)} for doc, score in context
                    ]
                },
                "output": target,
                "answerability": question_type
            }
            file.write(json.dumps(obj) + "\n")

In [111]:
dump_jsonl(training_data, BASE_DIR / "generator_reference" / "train_with_oversampling.jsonl")

In [109]:
dump_jsonl(train_data, BASE_DIR / "generator_reference" / "train.jsonl")

In [87]:
dev_raw = []
for data in dataset:
    obj = json.loads(data)
    conv_id = obj["conversation_id"]
    turn = int(obj["turn"])
    
    if(conv_id, turn) in dev_data.keys():
        dev_raw.append(obj)
with open(BASE_DIR / "generator_reference" / "dev.jsonl", "w") as file:
    for o in dev_raw:
        file.write(json.dumps(o) + "\n")

In [55]:
import ollama
model = "qwen2.5:7b"
def generate_answer(history, current, context):
    contexts_sorted = sorted(context, key=lambda x: x[1], reverse=True)
    formatted = []
    for i, (doc, score) in enumerate(contexts_sorted, start=1):
        formatted.append(
            f"[Document #{i} | Score={score:.3f}]\n{doc}\n"
        )
    context_with_score = "\n".join(formatted)
    history_text = "\n".join(f"- {h}" for h in history if h)
    prompt = f"""
            You are a RAG generator.
            Provide a FINAL ANSWER to the CURRENT question using the provided CONTEXT passages only.
            Use the information of CONTEXT base on the relative score.
            If CONTEXT is empty answer: "I don't know."
            If the answer is not in the context, answer: "I don't know."
            RULES:
            - Use ONLY facts in the CONTEXT.  
            - Do NOT add any external knowledge.  
            - Do NOT include phrases like "Answer:", "Final answer:", "Based on the context", etc.
            - Return ONLY the answer itself as one short paragraph or one sentence.  
            - No formatting (no **bold**, no lists, no markdown).

            
            CONTEXT:
            {context_with_score}
            
            HISTORY:
            {history_text}
            
            CURRENT QUESTION:
            {current}
            
            Answer:
            """.strip()
    resp = ollama.generate(model=model, prompt=prompt)
    return resp["response"]
    

In [56]:
try_context = data_dict[("dd6b6ffd177f2b311abe676261279d2f", 3)]["context"]
try_hist = data_dict[("dd6b6ffd177f2b311abe676261279d2f", 3)]["history"]
try_cur = data_dict[("dd6b6ffd177f2b311abe676261279d2f", 3)]["current"]
print(generate_answer(try_hist, try_cur, try_context))

Yes, the Arizona Cardinals were previously known as the Chicago Cardinals until they moved to St. Louis in 1959 and eventually to Arizona in 1988.


In [57]:
input_path = BASE_DIR / "generator_reference" / "reference.jsonl"
output_path = BASE_DIR / "generator_reference" / "output_with_pred.jsonl"
with open(input_path, "r") as fin, open(output_path, "w") as fout:
    for line in fin:
        obj = json.loads(line)


        history = [x["text"] for x in obj["input"]][:-1]
        current = [x["text"] for x in obj["input"]][-1]
        context = [(c["text"], c["score"]) for c in obj["contexts"]]

        # generator
        prediction = generate_answer(history, current, context)

        obj["predictions"] = [{"text": prediction}]

        fout.write(json.dumps(obj, ensure_ascii=False) + "\n")

In [89]:
import json
from collections import defaultdict

# ===========================
# Helper: load scores
# ===========================
def load_scores(path, drop_zero=False):
    """
    Returns:
      scores: dict {task_id: RB_agg}
      total: total samples with RB_agg
      dropped: number dropped due to RB_agg == 0
    """
    scores = {}
    total = 0
    dropped = 0

    with open(path) as f:
        for line in f:
            obj = json.loads(line)
            tid = obj.get("task_id")
            metrics = obj.get("metrics", {})

            if "RB_agg" not in metrics:
                continue

            total += 1
            score = metrics["RB_agg"][0]

            if drop_zero and score == 0:
                dropped += 1
                continue

            scores[tid] = score

    return scores, total, dropped


# ===========================
# Paths
# ===========================
BASELINE_PATH = "generator_reference/eval_gene_baseline.jsonl"
QWEN7_PATH    = "generator_reference/eval_dev_qwen2.57B_chatmode2.jsonl"
QWEN14_PATH   = "generator_reference/eval_dev_qwen3_14B_withoutLoRA3.jsonl"

DROP_ZERO = True   # ← 你可以切換 True / False


# ===========================
# Load all models
# ===========================
baseline_scores, base_total, base_dropped = load_scores(BASELINE_PATH, drop_zero=False)
qwen7_scores, q7_total, q7_dropped = load_scores(QWEN7_PATH, drop_zero=DROP_ZERO)
qwen14_scores, q14_total, q14_dropped = load_scores(QWEN14_PATH, drop_zero=DROP_ZERO)

# ===========================
# Align task_ids (VERY IMPORTANT)
# ===========================
common_ids = (
    set(baseline_scores.keys())
    & set(qwen7_scores.keys())
    & set(qwen14_scores.keys())
)

print("Common tasks:", len(common_ids))

# ===========================
# Compute averages
# ===========================
def avg(scores, ids):
    return sum(scores[i] for i in ids) / len(ids) if ids else 0.0

avg_base = avg(baseline_scores, common_ids)
avg_q7   = avg(qwen7_scores, common_ids)
avg_q14  = avg(qwen14_scores, common_ids)

# ===========================
# Print results
# ===========================
print("\n===== RB_agg Comparison =====")
print(f"Baseline avg RB_agg : {avg_base:.4f}")
print(f"Qwen2.5 7B avg     : {avg_q7:.4f}")
print(f"Qwen3   14B avg    : {avg_q14:.4f}")

print("\n===== Drop Statistics =====")
print(f"Baseline   total={base_total} dropped={base_dropped} drop_rate={base_dropped/base_total:.2%}")
print(f"Qwen2.5 7B total={q7_total} dropped={q7_dropped} drop_rate={q7_dropped/q7_total:.2%}")
print(f"Qwen3  14B total={q14_total} dropped={q14_dropped} drop_rate={q14_dropped/q14_total:.2%}")

print("\n===== Compared Samples =====")
print(len(common_ids))

Common tasks: 158

===== RB_agg Comparison =====
Baseline avg RB_agg : 0.3508
Qwen2.5 7B avg     : 0.4328
Qwen3   14B avg    : 0.4588

===== Drop Statistics =====
Baseline   total=842 dropped=0 drop_rate=0.00%
Qwen2.5 7B total=169 dropped=11 drop_rate=6.51%
Qwen3  14B total=169 dropped=11 drop_rate=6.51%

===== Compared Samples =====
158


In [57]:
bad_agg = 0
with open("generator_reference/eval_dev_qwen3_14B_withoutLoRA2.jsonl", "r") as f:
    for line in f:
        obj = json.loads(line)
        if obj["metrics"]["RB_agg"][0] < 0.4:
            bad_agg += 1
            print(obj["task_id"])
            print(obj["predictions"])
            print(obj["metrics"]["RB_agg"][0])
print(bad_agg)

5b2404d71f9ff7edabddb3b1a8b329e7<::>4
[{'text': 'California has more wildfires. The reference documents highlight that California experienced the largest wildfire season in its modern history in 2020, with over 4.2 million acres burned, and it continues to face significant wildfire activity, as evidenced by the number of wildfires in 2023.'}]
0.38453025385514056
535ebd306304982e4e49cc989ce5b10b<::>7
[{'text': "You're right that current EV batteries may not be ideal for a used car market due to their complexity and high cost. However, advancements in battery technology and the potential for removable and replaceable battery packs could make EVs more viable in the used car market in the future."}]
0.32151654163555116
535ebd306304982e4e49cc989ce5b10b<::>9
[{'text': 'Your concern is valid, as the resale value of EVs can be affected by battery longevity and market demand. However, advancements by companies like Toyota and GM suggest that battery durability is improving, which may help maint